In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error, confusion_matrix
from sklearn.neural_network import MLPClassifier

%matplotlib inline

In [2]:
data='/Users/alejandro/AnacondaProjects/Actuary Data/train_set.csv'
df = pd.read_csv(data, nrows=100000)

# dummy code claims
df['claim'] = df.Claim_Amount.apply(lambda x: 0 if x == 0 else 1)
df['Model_Year'] = df['Model_Year'] - np.min(df['Model_Year'])
df['Model_Year'] = df['Model_Year'].astype('float64')
df.head()

/anaconda3/envs/cv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Row_ID,Household_ID,Vehicle,Calendar_Year,Model_Year,Blind_Make,Blind_Model,Blind_Submodel,Cat1,Cat2,...,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount,claim
0,1,1,3,2005,24.0,K,K.78,K.78.2,D,C,...,0.261040,0.907793,-0.077998,M,-0.23153,-0.266117,-0.272337,-0.251419,0.0,0
1,2,2,2,2005,22.0,Q,Q.22,Q.22.3,B,C,...,0.432987,-0.726459,0.204785,O,-0.23153,-0.266117,-0.272337,-0.251419,0.0,0
2,3,3,1,2005,17.0,AR,AR.41,AR.41.1,B,?,...,-1.405797,-0.837048,-1.176858,F,-0.23153,-0.266117,-0.272337,-0.251419,0.0,0
3,4,3,1,2006,17.0,AR,AR.41,AR.41.1,B,?,...,-1.405797,-0.837048,-1.176858,F,-0.23153,-0.266117,-0.272337,-0.251419,0.0,0
4,5,3,2,2005,20.0,D,D.20,D.20.0,J,C,...,2.112691,1.534462,2.347260,F,-0.23153,-0.266117,-0.272337,-0.251419,0.0,0


In [3]:
X = df[['Cat1', 'Cat2', 'NVCat', 'Var5', 'Var6', 'Var7', 'Var8', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4', 'Vehicle', 'Model_Year', 'Blind_Model']]
y = df[['claim', 'Claim_Amount']].values

In [4]:
def pipeline(X):
    df_cont = pd.DataFrame()
    df_cat = pd.DataFrame()
    for x in X:
        if X[x].dtypes == 'float64':
            df_cont['{}'.format(x)] = X[x]
        else:
            df_cat['{}'.format(x)] = X[x]
    
    dummy = OneHotEncoder()
    dummyC = LabelEncoder()
    
    df_cat2 = np.zeros((df_cat.shape[0], 1))
    for x in df_cat:
        y = dummyC.fit_transform(df_cat[x].reshape(-1, 1))
        y = dummy.fit_transform(y.reshape(-1, 1)).toarray()
        y = pd.DataFrame(y[:, 1:])
        df_cat2 = np.hstack((df_cat2, y))
    df_cat = pd.DataFrame(df_cat2)
    
    df = pd.concat([df_cat, pd.DataFrame(df_cont)], axis=1)
    
    return df

In [5]:
X = pipeline(X)

/anaconda3/envs/cv/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  from ipykernel import kernelapp as app
/anaconda3/envs/cv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
model = RandomForestClassifier(random_state=42, class_weight='balanced')
model.fit(X, y[:,0])

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [7]:
y_probs = model.predict_proba(X)
y_probs = pd.DataFrame(y_probs)
y_probs = y_probs.loc[:,1].apply(lambda x: 1 if x > 0.005 else 0)
print('Accuracy:', accuracy_score(y_probs, y[:,0]))
tn, fp, fn, tp = confusion_matrix(y[:,0], y_probs).ravel()
print('Recall:', tp / (tp + fn))
pd.DataFrame(confusion_matrix(y[:,0], y_probs))

Accuracy: 0.758231
Recall: 0.999876619371


,0,1
0,750127,241768
1,1,8104


In [8]:
# model = RandomForestClassifier(random_state=42, class_weight='balanced')
model = MLPClassifier(alpha=0.0001, hidden_layer_sizes=512)
model.fit(X, y[:,0])

/anaconda3/envs/cv/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=512, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [9]:
y_probs = model.predict_proba(X)
y_probs = pd.DataFrame(y_probs)
y_probs = y_probs.loc[:,1].apply(lambda x: 1 if x > 0.005 else 0)
print('Accuracy:', accuracy_score(y_probs, y[:,0]))
tn, fp, fn, tp = confusion_matrix(y[:,0], y_probs).ravel()
print('Recall:', tp / (tp + fn))
pd.DataFrame(confusion_matrix(y[:,0], y_probs))

KeyboardInterrupt: 

In [ ]:
X2 = X.loc[y_probs == 1, :]
y2 = y[y_probs == 1, :]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model2 = DecisionTreeClassifier(random_state=42, class_weight='balanced')
model2.fit(X2, y2[:, 0])

In [ ]:
y_probs2 = model2.predict_proba(X2)
y_probs2 = pd.DataFrame(y_probs2)
y_probs2 = y_probs2.loc[:,1].apply(lambda x: 1 if x > 0.1 else 0)
print('Accuracy:', accuracy_score(y_probs2, y2[:, 0]))
tn, fp, fn, tp = confusion_matrix(y2[:, 0], y_probs2).ravel()
print('Recall:', tp / (tp + fn))
pd.DataFrame(confusion_matrix(y2[:, 0], y_probs2))

In [ ]:
X3 = X2.values[y_probs2 == 1, :]
y3 = y2[y_probs2 == 1, :]

In [ ]:
model3 = MLPClassifier(alpha=0.0001, hidden_layer_sizes=512)
model3.fit(X3, y3[:, 0])

In [ ]:
y_probs3 = model3.predict_proba(X3)
y_probs3 = pd.DataFrame(y_probs3)
y_probs3 = y_probs3.loc[:,1].apply(lambda x: 1 if x > 0.03 else 0)
print('Accuracy:', accuracy_score(y_probs3, y3[:, 0]))
tn, fp, fn, tp = confusion_matrix(y3[:, 0], y_probs3).ravel()
print('Recall:', tp / (tp + fn))
pd.DataFrame(confusion_matrix(y3[:, 0], y_probs3))

In [ ]:
X4 = X3[y_probs3 == 1, :]
y4 = y3[y_probs3 == 1, :]

In [ ]:
model4 = RandomForestClassifier(random_state=42, class_weight='balanced')
model4.fit(X4, y4[:, 0])

In [ ]:
y_probs4 = model4.predict_proba(X4)
y_probs4 = pd.DataFrame(y_probs4)
y_probs4 = y_probs4.loc[:,1].apply(lambda x: 1 if x > 0.15 else 0)
print('Accuracy:', accuracy_score(y_probs4, y4[:, 0]))
tn, fp, fn, tp = confusion_matrix(y4[:, 0], y_probs4).ravel()
print('Recall:', tp / (tp + fn))
pd.DataFrame(confusion_matrix(y4[:, 0], y_probs4))

In [ ]:
X5 = X4[y_probs4 == 1, :]
y5 = y4[y_probs4 == 1, :]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model5 = KNeighborsClassifier(20)
model5.fit(X5, y5[:, 0])

In [ ]:
y_probs5 = model5.predict_proba(X5)
y_probs5 = pd.DataFrame(y_probs5)
y_probs5 = y_probs5.loc[:,1].apply(lambda x: 1 if x > 0.05 else 0)
print('Accuracy:', accuracy_score(y_probs5, y5[:, 0]))
tn, fp, fn, tp = confusion_matrix(y5[:, 0], y_probs5).ravel()
print('Recall:', tp / (tp + fn))
pd.DataFrame(confusion_matrix(y5[:, 0], y_probs5))

In [ ]:
X6 = X5[y_probs5 == 1, :]
y6 = y5[y_probs5 == 1, :]

In [ ]:
model6 = DecisionTreeClassifier(random_state=42, class_weight='balanced')
model6.fit(X6, y6[:, 0])

In [ ]:
y_probs6 = model6.predict_proba(X6)
y_probs6 = pd.DataFrame(y_probs6)
y_probs6 = y_probs6.loc[:,1].apply(lambda x: 1 if x > 0.15 else 0)
print('Accuracy:', accuracy_score(y_probs6, y6[:, 0]))
tn, fp, fn, tp = confusion_matrix(y6[:, 0], y_probs6).ravel()
print('Recall:', tp / (tp + fn))
pd.DataFrame(confusion_matrix(y6[:, 0], y_probs6))

In [ ]:
X7 = X6[y_probs6 == 1, :]
y7 = y6[y_probs6 == 1, :]

In [ ]:
model7 = RandomForestClassifier(random_state=42, class_weight='balanced')
model7.fit(X7, y7[:, 0])

In [ ]:
y_probs7 = model7.predict_proba(X7)
y_probs7 = pd.DataFrame(y_probs7)
y_probs7 = y_probs7.loc[:,1].apply(lambda x: 1 if x > 0.07 else 0)
print('Accuracy:', accuracy_score(y_probs7, y7[:, 0]))
tn, fp, fn, tp = confusion_matrix(y7[:, 0], y_probs7).ravel()
print('Recall:', tp / (tp + fn))
pd.DataFrame(confusion_matrix(y7[:, 0], y_probs7))

In [ ]:
X8 = X7[y_probs7 == 1, :]
y8 = y7[y_probs7 == 1, :]

In [ ]:
model8 = RandomForestClassifier(random_state=42, class_weight='balanced')
model8.fit(X8, y8[:, 0])

In [ ]:
y_probs8 = model8.predict_proba(X8)
y_probs8 = pd.DataFrame(y_probs8)
y_probs8 = y_probs8.loc[:,1].apply(lambda x: 1 if x > 0.2 else 0)
print('Accuracy:', accuracy_score(y_probs8, y8[:, 0]))
tn, fp, fn, tp = confusion_matrix(y8[:, 0], y_probs8).ravel()
print('Recall:', tp / (tp + fn))
pd.DataFrame(confusion_matrix(y8[:, 0], y_probs8))

In [ ]:
X9 = X8[y_probs8 == 1, :]
y9 = y8[y_probs8 == 1, :]

In [ ]:
model9 = DecisionTreeClassifier(random_state=42)
model9.fit(X9, y9[:, 0])

In [ ]:
y_probs9 = model9.predict_proba(X9)
y_probs9 = pd.DataFrame(y_probs9)
y_probs9 = y_probs9.loc[:,1].apply(lambda x: 1 if x > 0.13 else 0)
print('Accuracy:', accuracy_score(y_probs9, y9[:, 0]))
tn, fp, fn, tp = confusion_matrix(y9[:, 0], y_probs9).ravel()
print('Recall:', tp / (tp + fn))
pd.DataFrame(confusion_matrix(y9[:, 0], y_probs9))

In [ ]:
X10 = X9[y_probs9 == 1, :]
y10 = y9[y_probs9 == 1, :]

In [ ]:
model10 = RandomForestClassifier(random_state=42)
model10.fit(X10, y10[:, 0])

In [ ]:
y_probs10 = model10.predict_proba(X10)
y_probs10 = pd.DataFrame(y_probs10)
y_probs10 = y_probs10.loc[:,1].apply(lambda x: 1 if x > 0.1 else 0)
print('Accuracy:', accuracy_score(y_probs10, y10[:, 0]))
tn, fp, fn, tp = confusion_matrix(y10[:, 0], y_probs10).ravel()
print('Recall:', tp / (tp + fn))
pd.DataFrame(confusion_matrix(y10[:, 0], y_probs10))